# Description

Tutorial on how to read from the scBaseCamp TileDB-SOMA database

# Set up

[tiledbsoma-py](https://pypi.org/project/tiledbsoma/) must be installed

In [1]:
# set the path to the database (local or cloud)
db_uri = "/scratch/multiomics/nickyoungblut/tiledb-loader/tiledb-soma_GeneFull_Ex50pAS"

In [39]:
# load packages
import numpy as np
import pandas as pd
import tiledbsoma
import tiledbsoma.io

# Observations

In [4]:
# get obs colnames
with tiledbsoma.open(db_uri) as exp:
    print(exp.obs.schema)

soma_joinid: int64 not null
obs_id: large_string
gene_count: int64
umi_count: float
barcode: large_string
SRX_accession: dictionary<values=string, indices=int32, ordered=0>
lib_prep: dictionary<values=string, indices=int32, ordered=0>
tech_10x: dictionary<values=string, indices=int32, ordered=0>
organism: dictionary<values=string, indices=int32, ordered=0>
tissue: dictionary<values=string, indices=int32, ordered=0>
disease: dictionary<values=string, indices=int32, ordered=0>
purturbation: dictionary<values=string, indices=int32, ordered=0>
cell_line: dictionary<values=string, indices=int32, ordered=0>
czi_collection_id: dictionary<values=string, indices=int32, ordered=0>
czi_collection_name: dictionary<values=string, indices=int32, ordered=0>


In [37]:
# view all metadata for the first 3 observations
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.obs.read()
        .concat()
        .slice(0,3) 
        .to_pandas()
    )
pd.set_option('display.max_columns', 100)
df

,soma_joinid,obs_id,gene_count,umi_count,barcode,SRX_accession,lib_prep,tech_10x,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name
0,0,AAACCCAAGGGAGATA_SRX10681588,1677,4085.0,AAACCCAAGGGAGATA,SRX10681588,10x_Genomics,3_prime_gex,human,blood,not specified,lean,not applicable,None,None
1,1,AAACCCAAGGTTGGAC_SRX10681588,1371,3866.0,AAACCCAAGGTTGGAC,SRX10681588,10x_Genomics,3_prime_gex,human,blood,not specified,lean,not applicable,None,None
2,2,AAACCCACAGTGGTGA_SRX10681588,1891,3936.0,AAACCCACAGTGGTGA,SRX10681588,10x_Genomics,3_prime_gex,human,blood,not specified,lean,not applicable,None,None


In [8]:
# count total observations
with tiledbsoma.open(db_uri) as exp:
    total_cells = (
        exp.obs.read(column_names=["obs_id"])
        .concat()
        .group_by([])
        .aggregate([
            ([], 'count_all'),
        ])
        .to_pandas()["count_all"].values[0]
    )
print(f"Total cells: {total_cells}")

Total cells: 48393197


In [10]:
# observations per dataset
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.obs.read(column_names=["obs_id", "SRX_accession"])
        .concat()
        .group_by(["SRX_accession"])
        .aggregate([
            ([], 'count_all'),
        ])
        .sort_by([("count_all", "descending")])
        .to_pandas()
    )
df

,SRX_accession,count_all
0,SRX22915751,100908
1,SRX17521047,93266
2,SRX18774274,87650
3,SRX18899905,83021
4,SRX17917753,78088
...,...,...
6477,SRX16742101,68
6478,SRX16742118,67
6479,SRX11218626,56
6480,SRX16742131,55


## Metadata

### Summary

In [17]:
# obs count per 10X Genomics technology
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.obs.read()
        .concat()
        .group_by(["tech_10x"])
        .aggregate([
            ([], 'count_all'),
        ])
        .sort_by([("count_all", "descending")])
        .to_pandas()
    )
df

,tech_10x,count_all
0,3_prime_gex,36420130
1,5_prime_gex,8478561
2,multiome,1164951
3,vdj,699731
4,feature_barcoding,560797
5,not_applicable,483986
6,other,481171
7,cellplex,103870


In [18]:
# obs count per 10X Genomics technology
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.obs.read()
        .concat()
        .group_by(["organism"])
        .aggregate([
            ([], 'count_all'),
        ])
        .sort_by([("count_all", "descending")])
        .to_pandas()
    )
df

,organism,count_all
0,human,25110227
1,mouse,23282970


### Query

In [20]:
# create metadata query
obs_query = tiledbsoma.AxisQuery(value_filter='organism in ["human"]')

# obs count per 10X Genomics technology for the query results
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.axis_query("RNA", obs_query=obs_query)
        .obs()
        .concat()
        .group_by(["organism", "tech_10x"])
        .aggregate([
            ([], 'count_all'),
        ])
        .sort_by([("count_all", "descending")])
        .to_pandas()
    )
df

,organism,tech_10x,count_all
0,human,3_prime_gex,16933933
1,human,5_prime_gex,6086171
2,human,multiome,583889
3,human,vdj,533631
4,human,feature_barcoding,478525
5,human,other,261601
6,human,not_applicable,206333
7,human,cellplex,26144


In [31]:
# create metadata query
obs_query = tiledbsoma.AxisQuery(
    value_filter='organism in ["human"] and tech_10x in ["cellplex"] and gene_count >= 1000'
)

# gene and umi counts
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.axis_query("RNA", obs_query=obs_query)
        .obs()
        .concat()
        .select(["SRX_accession", "gene_count", "umi_count"])
        .sort_by([("gene_count", "descending")])
        .to_pandas()
    )
df

,SRX_accession,gene_count,umi_count
0,SRX22369542,14245,272316.0
1,SRX22369542,14093,272180.0
2,SRX22369542,14056,282893.0
3,SRX22369542,14018,253654.0
4,SRX22369542,13895,265843.0
...,...,...,...
18153,SRX23560128,1000,2307.0
18154,SRX23560128,1000,1436.0
18155,SRX23560128,1000,2062.0
18156,SRX23560131,1000,1890.0


# Variables

In [13]:
# get colnames
with tiledbsoma.open(db_uri) as exp:
    print(exp.ms["RNA"].var.schema)

soma_joinid: int64 not null
var_id: large_string


In [16]:
# read in the var level data
with tiledbsoma.Experiment.open(db_uri) as exp:
    df = (
        exp.ms["RNA"]
        .var.read(column_names=["soma_joinid", "var_id"])
        .concat()
        .to_pandas()
    )
df

,soma_joinid,var_id
0,0,ENSG00000000003
1,1,ENSG00000000005
2,2,ENSG00000000419
3,3,ENSG00000000457
4,4,ENSG00000000460
...,...,...
68881,68881,ENSMUSG00000118552
68882,68882,ENSMUSG00000118553
68883,68883,ENSMUSG00000118554
68884,68884,ENSMUSG00000118560


# Counts

## Slice

In [38]:
# sparse count matrix for the first 3 observations
with tiledbsoma.Experiment.open(db_uri) as exp:
    print(
        exp.ms["RNA"].X["data"]
        .read((slice(0,3),))
        .coos()
        .concat() 
    )

<pyarrow.SparseCOOTensor>
type: float
shape: (52967109, 68886)


In [41]:
# Get gene count per obs
def get_genes_per_obs(db_uri, start, end):
    with tiledbsoma.Experiment.open(db_uri) as exp:
        data = (
            exp.ms["RNA"].X["data"]
            .read((slice(start, end),)).coos().concat()
        )
    return np.diff(data.to_scipy().tocsr().indptr)[start:end]

## first 10 observations
get_genes_per_obs(db_uri, 0, 10)

array([1677, 1371, 1891, 1967, 2050, 1303, 3151, 1705, 1276, 1507],
      dtype=int32)

In [43]:
# Get UMI count per obs
def get_umi_counts_per_obs(db_uri, start, end):
    with tiledbsoma.Experiment.open(db_uri) as exp:
        data = exp.ms["RNA"].X["data"].read((slice(start, end),)).coos().concat()
    sp = data.to_scipy().tocsr()
    return sp.sum(axis=1).A1[start:end]

## first 10 observations
get_umi_counts_per_obs(db_uri, 0, 10)

array([4085., 3866., 3936., 5528., 6013., 2288., 7996., 4050., 3031.,
       4726.], dtype=float32)

# 